# Merging Datasets for Geolocation Analysis and Feature engineering  

In [1]:
import pandas as pd
import numpy as np

class FraudPreprocessor:
    def __init__(self, fraud_path, ip_path):
        self.fraud_path = fraud_path
        self.ip_path = ip_path
        self.fraud_df = None
        self.ip_df = None
        self.merged_df = None

    def load_data(self):
        self.fraud_df = pd.read_csv(self.fraud_path)
        self.ip_df = pd.read_csv(self.ip_path)
        return self

    def preprocess_ips(self):
        # Convert IPs in both datasets to int (they are already numeric, just ensure type)
        self.fraud_df['ip_int'] = self.fraud_df['ip_address'].astype(np.int64)
        self.ip_df['lower_bound_ip_int'] = self.ip_df['lower_bound_ip_address'].astype(np.int64)
        self.ip_df['upper_bound_ip_int'] = self.ip_df['upper_bound_ip_address'].astype(np.int64)
        return self

    def merge_geolocation(self):
        # Sort for asof merge
        self.fraud_df = self.fraud_df.sort_values('ip_int')
        self.ip_df = self.ip_df.sort_values('lower_bound_ip_int')

        merged = pd.merge_asof(
            self.fraud_df,
            self.ip_df,
            left_on='ip_int',
            right_on='lower_bound_ip_int',
            direction='backward'
        )

        # Filter valid IP ranges
        self.merged_df = merged[merged['ip_int'] <= merged['upper_bound_ip_int']].copy()
        return self

    def engineer_time_features(self):
        df = self.merged_df

        df['signup_time'] = pd.to_datetime(df['signup_time'])
        df['purchase_time'] = pd.to_datetime(df['purchase_time'])
        df['time_since_signup'] = (df['purchase_time'] - df['signup_time']).dt.total_seconds() / 3600.0  # in hours
        df['hour_of_day'] = df['purchase_time'].dt.hour
        df['day_of_week'] = df['purchase_time'].dt.dayofweek

        self.merged_df = df
        return self

    def engineer_transaction_features(self):
        df = self.merged_df.sort_values(['user_id', 'purchase_time'])
        df['tx_count_by_user'] = df.groupby('user_id')['purchase_time'].transform('count')
        df['prev_purchase_time'] = df.groupby('user_id')['purchase_time'].shift(1)
        df['txn_velocity_hours'] = (
            (df['purchase_time'] - df['prev_purchase_time']).dt.total_seconds() / 3600.0
        )
        df['txn_velocity_hours'] = df['txn_velocity_hours'].fillna(-1)
        self.merged_df = df
        return self

    def save(self, save_path):
        self.merged_df.to_csv(save_path, index=False)
        print(f"Engineered data saved to {save_path}")

    def run_all(self, save_path='../data/cleaned/fraud_data_engineered.csv'):
        self.load_data()\
            .preprocess_ips()\
            .merge_geolocation()\
            .engineer_time_features()\
            .engineer_transaction_features()
        self.save(save_path)
        print(self.merged_df[[
            'user_id', 'ip_address', 'country', 'hour_of_day', 'day_of_week',
            'time_since_signup', 'tx_count_by_user', 'txn_velocity_hours'
        ]].head())
        return self.merged_df

# Example usage:
preprocessor = FraudPreprocessor(
    fraud_path='../data/cleaned/fraud_data_cleaned.csv',
    ip_path='../data/cleaned/ip_country_cleaned.csv'
)
engineered_df = preprocessor.run_all()

Engineered data saved to ../data/cleaned/fraud_data_engineered.csv
        user_id    ip_address        country  hour_of_day  day_of_week  \
31545         2  8.802175e+08  United States           10            5   
97542         4  2.785906e+09    Switzerland           21            5   
12873         8  3.560567e+08  United States           11            3   
104500       12  2.985180e+09         Mexico           20            2   
21011        16  5.783125e+08  United States           12            3   

        time_since_signup  tx_count_by_user  txn_velocity_hours  
31545          990.273333                 1                -1.0  
97542         2788.855278                 1                -1.0  
12873         1852.000278                 1                -1.0  
104500        1286.523611                 1                -1.0  
21011          886.966667                 1                -1.0  


# Data Transformation

In [5]:
# Check your data structure
engineered_df = pd.read_csv('../data/cleaned/fraud_data_engineered.csv')
print("DataFrame info:")
print(engineered_df.info())
print("\nFirst few rows:")
print(engineered_df.head())
print("\nColumn dtypes:")
print(engineered_df.dtypes)

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129146 entries, 0 to 129145
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   user_id                 129146 non-null  int64  
 1   signup_time             129146 non-null  object 
 2   purchase_time           129146 non-null  object 
 3   purchase_value          129146 non-null  int64  
 4   device_id               129146 non-null  object 
 5   source                  129146 non-null  object 
 6   browser                 129146 non-null  object 
 7   sex                     129146 non-null  object 
 8   age                     129146 non-null  int64  
 9   ip_address              129146 non-null  float64
 10  class                   129146 non-null  int64  
 11  ip_int                  129146 non-null  int64  
 12  lower_bound_ip_address  129146 non-null  float64
 13  upper_bound_ip_address  129146 non-null  float64
 14  coun

In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

class FraudDataTransformer:
    def __init__(
        self,
        target_column='class',
        scaling_method='standard',
        sampling_method='smote',
        test_size=0.2,
        random_state=42,
        categorical_cols=None,
        exclude_cols=None,
        max_categories=50  # Limit categories for high-cardinality columns
    ):
        self.target_column = target_column
        self.scaling_method = scaling_method
        self.sampling_method = sampling_method
        self.test_size = test_size
        self.random_state = random_state
        self.categorical_cols = categorical_cols
        self.exclude_cols = exclude_cols or []
        self.max_categories = max_categories
        self.numerical_cols = None
        self.pipeline = None
        self.preprocessor = None

    def _get_scaler(self):
        if self.scaling_method == 'standard':
            return StandardScaler()
        elif self.scaling_method == 'minmax':
            return MinMaxScaler()
        else:
            raise ValueError("Choose 'standard' or 'minmax' for scaling_method.")

    def _get_sampler(self):
        if self.sampling_method == 'smote':
            return SMOTE(random_state=self.random_state)
        elif self.sampling_method == 'undersample':
            return RandomUnderSampler(random_state=self.random_state)
        elif self.sampling_method == 'none':
            return 'passthrough'
        else:
            raise ValueError("Choose 'smote', 'undersample', or 'none' for sampling_method.")

    def _identify_columns(self, X):
        """Identify categorical and numerical columns more robustly"""
        # Always exclude these columns from numerical processing
        always_exclude = ['signup_time', 'purchase_time', 'prev_purchase_time', 'ip_int', 'lower_bound_ip_int', 'upper_bound_ip_int']
        
        # If user specified categorical columns, use them
        if self.categorical_cols is not None:
            categorical = self.categorical_cols
        else:
            # Auto-detect categorical columns
            categorical = X.select_dtypes(include=['object', 'category']).columns.tolist()
        
        # Numerical columns are all others except categorical and excluded
        all_excluded = categorical + self.exclude_cols + always_exclude
        numerical = [col for col in X.columns if col not in all_excluded]
        
        # Only include truly numerical columns
        numerical = [col for col in numerical if X[col].dtype in ['int64', 'float64']]
        
        return categorical, numerical

    def _build_preprocessor(self, X):
        self.categorical_cols, self.numerical_cols = self._identify_columns(X)
        
        print(f"Categorical columns: {self.categorical_cols}")
        print(f"Numerical columns: {self.numerical_cols}")
        
        transformers = []
        
        if self.numerical_cols:
            transformers.append(('num', self._get_scaler(), self.numerical_cols))
        
        if self.categorical_cols:
            # Use sparse=True to save memory
            transformers.append(('cat', OneHotEncoder(
                handle_unknown='ignore', 
                sparse=True,  # Use sparse matrix
                max_categories=self.max_categories  # Limit categories
            ), self.categorical_cols))

        self.preprocessor = ColumnTransformer(
            transformers=transformers,
            sparse_threshold=0.3  # Use sparse output if more than 30% sparse
        )
        return self.preprocessor

    def transform(self, df):
        # 1. Split features and target
        X = df.drop(columns=[self.target_column])
        y = df[self.target_column]

        # 2. Train-test split (stratified)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.test_size, stratify=y, random_state=self.random_state
        )

        print("\n�� Class distribution BEFORE sampling:")
        print(y_train.value_counts(normalize=True).rename("proportion"))

        # 3. Build preprocessing pipeline
        preprocessor = self._build_preprocessor(X_train)
        sampler = self._get_sampler()

        # 4. Define full transformation pipeline
        self.pipeline = ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('sampler', sampler)
        ])

        # 5. Fit-transform on training set
        X_train_resampled, y_train_resampled = self.pipeline.fit_resample(X_train, y_train)

        # 6. Transform test set (only preprocessing, no sampling)
        X_test_transformed = self.preprocessor.transform(X_test)

        print("\n✅ Class distribution AFTER sampling:")
        print(pd.Series(y_train_resampled).value_counts(normalize=True).rename("proportion"))

        return X_train_resampled, X_test_transformed, y_train_resampled, y_test

# --- Example usage ---

# Load your engineered data
engineered_df = pd.read_csv('../data/cleaned/fraud_data_engineered.csv')

# Specify your categorical columns (adjust as needed for your data)
# Remove device_id if it has too many unique values
categorical_cols = ['country', 'browser', 'source', 'sex']  # Removed device_id

# If your target column is named 'class'
transformer = FraudDataTransformer(
    target_column='class',
    scaling_method='standard',      # or 'minmax'
    sampling_method='smote',        # or 'undersample' or 'none'
    test_size=0.2,
    random_state=42,
    categorical_cols=categorical_cols,
    max_categories=50  # Limit categories to prevent memory issues
)

X_train_res, X_test_trans, y_train_res, y_test = transformer.transform(engineered_df)


�� Class distribution BEFORE sampling:
class
0    0.90501
1    0.09499
Name: proportion, dtype: float64
Categorical columns: ['country', 'browser', 'source', 'sex']
Numerical columns: ['user_id', 'purchase_value', 'age', 'ip_address', 'lower_bound_ip_address', 'upper_bound_ip_address', 'time_since_signup', 'hour_of_day', 'day_of_week', 'tx_count_by_user', 'txn_velocity_hours']


/home/dinki/Desktop/KAIMProjects/week-8/fraud-detection-ecommerce-banking/venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



✅ Class distribution AFTER sampling:
class
0    0.5
1    0.5
Name: proportion, dtype: float64
